## Attention location shift

The paper represented the shift using this formula:

$$\widetilde{w}_t(i) = \sum^{N-1}_{j=0} w^g_t(j)\cdot s_t(i-j)$$

This assumes that the shift from the content addressing can shift from the current location to $N$ slots away, where $N$ is the size of the memory slots.

However, if you restrict the shifts to be -1 and +1 from the current location, then $j$ goes from -1 to 1. Another way to think about it is that for $-1 \leq j \leq 1$  $w^g_t(j)$ sums to 1, and is 0 everywhere else. This is all under the assumption that $i - j$ wraps around.

My implementation of this in Theano was to use an indexing trick to do a sort of convolution over the attention distribution. This works by first getting a matrix of this form:

In [4]:
import numpy as np
import scipy.linalg
idxs = np.arange(10)
idxs

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [12]:
conv_idxs = scipy.linalg.circulant(idxs).T
conv_idxs[np.arange(-1,2)]

array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [9, 0, 1, 2, 3, 4, 5, 6, 7, 8]])

The ``scipy.linalg.circulant`` gives a circulant matrix of $N$ x $N$ given a size $N$ input vector. It basically rotates the same vector around $N$ steps.

This gives you an index over the attention you are trying to modify, so indexing a vector using a index matrix such as this gives you a new matrix of the same shape as the index matrix. As an example:

In [25]:
data = np.random.randint(10,size=10)
windowed_idxs = conv_idxs[np.arange(-1,2)]
print data
print
print data[windowed_idxs]

[7 8 0 5 7 5 7 6 7 8]

[[8 0 5 7 5 7 6 7 8 7]
 [7 8 0 5 7 5 7 6 7 8]
 [8 7 8 0 5 7 5 7 6 7]]


So now that we have a matrix that scans over the previous attention, the shifted attention is just a matrix multiplication between the output from the shift part of the head, and this new matrix. Again, I rely heavily on Theano to give the right gradient calculations for this, so this works well, and I can then generalise this to larger windows easily.